### Wstęp do Uczenia Maszynowego 
##### Laboratorium 05

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import recall_score, precision_score, roc_auc_score, roc_curve

-----
##### *Zadanie 1*
------
Wczytaj dane `pima.csv`.

a) Podziel dane za część treningową i testową (3:2).

b) Dopasuj model drzewa do danych treningowych.

c) Dopasuj model regresji logistycznej do danych treningowych.

d) Podaj miary: czułość, precyzja, AUC dla obu modeli na zbiorze testowym.

e) Narysuj krzywą ROC dla obu modeli na zbiorze testowym.

In [ ]:
# Wczytanie danych
pima = pd.read_csv("../lab02/pima.csv")

In [ ]:
# Podział na X i y
y = pima.Outcome
X = pima.drop(['Outcome'], axis = 1)

# Podział na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

# Model drzewa bez optymalizacji hiperparametrów
tree_default = DecisionTreeClassifier()
tree_default.fit(X_train, y_train)

# Model drzewa z wybranymi hiperparametrami
tree = DecisionTreeClassifier(max_depth=7, min_samples_leaf=10)
tree.fit(X_train, y_train)

# Model regresji logistycznej
glm = LogisticRegression(penalty=None, max_iter = 500)
glm.fit(X_train, y_train)

In [ ]:
# Predykcja i wyliczenie metryk
y_pred_tree_default = tree_default.predict(X_test)
y_pred_tree = tree.predict(X_test)
y_pred_glm = glm.predict(X_test)


print("Model DRZEWA DEFAULT")
print("---------------------------")
print("precision:", np.round(precision_score(y_test, y_pred_tree_default), 3))
print("recall:", np.round(recall_score(y_test, y_pred_tree_default), 3))
print("auc:", )
print("---------------------------")

print("Model DRZEWA")
print("---------------------------")
print("precision:", np.round(precision_score(y_test, y_pred_tree), 3))
print("recall:", np.round(recall_score(y_test, y_pred_tree), 3))
print("---------------------------")

print("Model REGRESJI LOGISTYCZNEJ")
print("---------------------------")
print("precision", np.round(precision_score(y_test, y_pred_glm), 3))
print("recall:", np.round(recall_score(y_test, y_pred_glm), 3))
print("---------------------------")

In [ ]:
# AUC i krzywa ROC
pred_tree_default = tree_default.predict_proba(X_test)
pred_tree = tree.predict_proba(X_test)
pred_glm = glm.predict_proba(X_test)

fpr, tpr, thresholds = roc_curve(y_test, pred_tree_default[:,1])
plt.plot(fpr,tpr,label="Tree DEFAULT, AUC="+str(round(roc_auc_score(y_test, pred_tree_default[:,1]), 4)))
fpr, tpr, thresholds = roc_curve(y_test, pred_tree[:,1])
plt.plot(fpr,tpr,label="Tree, AUC="+str(round(roc_auc_score(y_test, pred_tree[:,1]), 4)))
fpr, tpr, thresholds = roc_curve(y_test, pred_glm[:,1])
plt.plot(fpr,tpr,label="LM, AUC="+str(round(roc_auc_score(y_test, pred_glm[:,1]), 4)))
plt.legend()

-----
##### *Zadanie 2*
------
Eksperyment symulacyjny. Wygenerujmy dane w następujący sposób. Niech $n = 200$, $p = 5$. Wektor $\beta_0 = 1$, $\beta_1 = [2, 1, 0.5, 0.01, 0]$, $x_i \sim N_p(0, I)$ dla $i = 1,2, \dots, n$. $y_i$ pochodzi z rozkładu $Bern(p_i)$, gdzie $p_i=\frac{exp(\beta_0 + x_{i\cdot}\beta_1)}{1 + exp(\beta_0 + x_{i\cdot}\beta_1)}$.

a) Dopasuj model regresji logistycznej. Porównaj prawdziwe wartości wektora $\beta$ z wyestymowanymi.

b) Dopasuj model regresji logistycznej z regularyzacją $L2$ i współczynnikami $C = (10, 5, 2, 1, 0.5, 0.1, 0.01, 0.005)$. jak zmienieją się współczynniki wraz ze wzrostem współczynnika lambda?

c) Dopasuj model regresji logistycznej z regularyzacją $L1$ i współczynnikami $C = (10, 5, 2, 1, 0.5, 0.1, 0.01, 0.005)$. jak zmienieją się współczynniki wraz ze wzrostem współczynnika lambda?

d) Powtórz eksperyment 500 razy dla modelu logistycznego bez regularyzacji, z regularyzacją $L2$ i parametrem $C=1$ i regularyzacją $L1$ i parametrem $C=1$. Oblicz MSE.

In [ ]:
# Symulacja danych wejściowych
n = 200 # liczba obserwacji (wierszy)
p = 5 # liczba zmiennych (kolumn)
beta_0 = 1 # wyraz wolny
beta_1 = np.array([2, 1, 0.5, 0.01, 0]) # współczynniki
X = np.random.normal(0, 1, size = (n, p)) # losowanie z rozkładu normalnego wielowymiarowego
Xbeta = X@beta_1 + beta_0 
probs = np.exp(Xbeta)/(1 + np.exp(Xbeta)) # p_i
y = np.random.binomial(1, probs) # wartości y


In [ ]:
# a)
# model regresji logistycznej bez regularyzacji
lm = LogisticRegression(penalty=None)
lm.fit(X, y)
print(lm.coef_)
print(lm.intercept_)

In [ ]:
# b) 
coefs = []
intercepts = []
C = np.array([10, 5, 2, 1, 0.5, 0.1, 0.01, 0.005]) # zakres wartości C z polecenia

for c in C:
    # model regresji logistycznej z karą L2
    glm = LogisticRegression(penalty="l2", C = c)
    glm.fit(X, y)
    coefs.append(glm.coef_[0]) # zapisuję współczynniki
    intercepts.append(glm.intercept_[0]) # zapisuję wyraz wolny

In [ ]:
# wykres zależności parametru C (1/lambda) od wartości współczynników
plt.plot(1/C, np.array(coefs), label = ['B1', 'B2', 'B3', 'B4', 'B5'])
plt.legend()

In [ ]:
# c) 
coefs = []
intercepts = []
C = np.array([10, 5, 2, 1, 0.5, 0.1, 0.01, 0.005]) # zakres wartości C z polecenia

for c in C:
    # model regresji logistycznej z karą L1, należy zmieć "solver"
    glm = LogisticRegression(penalty="l1", C = c, solver = 'liblinear')
    glm.fit(X, y)
    coefs.append(glm.coef_[0]) # zapisuję współczynniki
    intercepts.append(glm.intercept_[0]) # zapisuję wyraz wolny

In [ ]:
# wykres zależności parametru C (1/lambda) od wartości współczynników
plt.plot(1/C, np.array(coefs), label = ['B1.1', 'B1.2', 'B1.3', 'B1.4', 'B1.5'])
plt.legend()

In [ ]:
# d)
k = 500 # liczba eksperymentów 
n = 200
p = 5
beta_0 = 1
beta_1 = np.array([2, 1, 0.5, 0.01, 0])

# miejsce do zapisania wyników
MSE = []
MSE_l1 = []
MSE_l2 = []

for i in range(k):
    # generowanie danych    
    X = np.random.normal(0, 1, size = (n, p))
    Xbeta = X@beta_1+beta_0
    probs = np.exp(Xbeta)/(1+np.exp(Xbeta)) 
    y = np.random.binomial(1, probs)
    
    # model regresji logistycznej bez regularyzacji
    lm = LogisticRegression(penalty = None, max_iter = 100)
    lm.fit(X,y)

    MSE.append(np.sum((lm.coef_ - beta_1)**2 + (lm.intercept_ - beta_0)**2))
    
    # model regresji logistycznej z karą L1
    lm_l1 = LogisticRegression(penalty = 'l1', max_iter = 100, solver = 'liblinear')
    lm_l1.fit(X,y)

    MSE_l1.append(np.sum((lm_l1.coef_ - beta_1)**2 + (lm_l1.intercept_ - beta_0)**2))

    # model regresji logistycznej z karą L2
    lm_l2 = LogisticRegression(penalty = 'l2', max_iter = 100)
    lm_l2.fit(X,y)
    
    MSE_l2.append(np.sum((lm_l2.coef_ - beta_1)**2 + (lm_l2.intercept_ - beta_0)**2))

print('MSE:', np.round(np.mean(MSE), 5))
print('MSE l1:', np.round(np.mean(MSE_l1), 5))
print('MSE l2:', np.round(np.mean(MSE_l2), 5))

-----
##### *Zadanie 3*
------
Wpływ skalowania zmiennych. Dopasuj modele regresji logistycznej
1. z regularyzacją L2 dla danych bez skalowania i dla danych ze skalowaniem zmiennych (StandardScaler);
2. z regularyzacją L1 dla danych bez skalowania i dla danych ze skalowaniem zmiennych (StandardScaler).

Porównaj współczynniki.

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import pandas as pd

# Wczytanie danych
credit = fetch_openml(name='credit-g', version=1, as_frame=True)
df = credit.frame

print(df.shape)
# wybranie kilku kolumn
sel_cols = ['age', 'existing_credits', 'duration', 'credit_amount', 'savings_status', 'credit_history', 'class']
df = df[sel_cols]
df.head(10)

In [ ]:
# kodowanie klas
df['class'] = (df['class'] == 'good').astype(int)

# wybór X i y
X = df.drop(columns='class')
y = df['class']

In [ ]:
# kodowanie zmiennych kategorycznych
X_dummies = pd.get_dummies(X, drop_first=True)

1. z regularyzacją L2 dla danych bez skalowania i dla danych ze skalowaniem zmiennych (StandardScaler)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

# podział na próbkę treningową i testową
X_train, X_test, y_train, y_test = train_test_split(
    X_dummies, y, test_size=0.3, random_state=42, stratify=y
)

# model regresji logistycznej na danych bez skalowanie 
model_raw = LogisticRegression(penalty='l2', C=1.0, solver='lbfgs', max_iter=5000)
model_raw.fit(X_train, y_train)

# predykcja
y_pred_raw = model_raw.predict(X_test)
# auc
auc_raw = roc_auc_score(y_test, model_raw.predict_proba(X_test)[:,1])
print("AUC bez skalowania:", auc_raw)


In [ ]:
# nazwy kolumn z wartością współczynników
coef_raw = pd.Series(model_raw.coef_[0], index=X.columns)
coef_raw.sort_values(ascending=False).head(10)


In [ ]:
from sklearn.preprocessing import StandardScaler

# skalowanie zmiennych
scaler = StandardScaler()
# X_train_s = scaler.fit_transform(X_train)
# X_test_s = scaler.transform(X_test)

X_s = scaler.fit_transform(X)


X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(
    X_s, y, test_size=0.3, random_state=42, stratify=y
)


model_scaled = LogisticRegression(penalty='l2', C=1.0, solver='lbfgs', max_iter=5000)
model_scaled.fit(X_train_s, y_train_s)

y_pred_s = model_scaled.predict(X_test_s)
auc_scaled = roc_auc_score(y_test_s, model_scaled.predict_proba(X_test_s)[:,1])
print("AUC po skalowaniu:", auc_scaled)


In [ ]:
# nazwy kolumn z wartością współczynników
coef_scaled = pd.Series(model_scaled.coef_[0], index=X.columns)

# porównanie współczynników bez skalowanie i ze skalowaniem
comparison = pd.DataFrame({
    'coef_raw': coef_raw,
    'coef_scaled': coef_scaled
})


In [ ]:
# wykres ukazujący różnice w wartościach współczynników dla danych ze skalowaniem i bez
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(10,5))
plt.bar(np.arange(len(coef_raw)), coef_raw, width=0.4, label='bez skalowania')
plt.bar(np.arange(len(coef_scaled))+0.4, coef_scaled, width=0.4, label='po skalowaniu')
plt.xlabel("Indeks cechy")
plt.ylabel("Wartość współczynnika β")
plt.title("Wpływ skalowania na współczynniki regresji logistycznej (Credit-G)")
plt.legend()
plt.show()

2. z regularyzacją L1 dla danych bez skalowania i dla danych ze skalowaniem zmiennych (StandardScaler)

In [ ]:
# model regresji logistycznej na danych bez skalowanie 
model_l1_raw = LogisticRegression(penalty='l1', solver='saga', C=0.5, max_iter=5000)
model_l1_raw.fit(X_train, y_train)
coef_l1_raw = pd.Series(model_l1_raw.coef_[0], index=X.columns)
print("Niezerowe współczynniki (bez skalowania):", sum(coef_l1_raw != 0)) # liczba niezerowych współczynników


In [ ]:
# model regresji logistycznej na danych zez skalowaniem
model_l1_scaled = LogisticRegression(penalty='l1', solver='saga', C=0.5, max_iter=5000)
model_l1_scaled.fit(X_train_s, y_train_s)
coef_l1_scaled = pd.Series(model_l1_scaled.coef_[0], index=X.columns)
print("Niezerowe współczynniki (po skalowaniu):", sum(coef_l1_scaled != 0)) # liczba niezerowych współczynników


In [ ]:
# porównanie wartości współczynników
comparison = pd.DataFrame({
    'coef_raw': coef_l1_raw,
    'coef_scaled': coef_l1_scaled
})
comparison